# Data Programming

## Introduction and context
This is a data analysis of the academic journal frontiers. We are going to take a high level look at the journal overall and a more focused look at specific topics and articles. 

Specifically this analysis is interested in academic work related to artifical intelligence, machine learning, neuroscience, computational biology, cognitive science and psychology.

## Report contents
- Inroduction and context
- Brief description of data
- High level description of data
    - Where it has come from 
    - Why is this data relevant
- Summary of key findings and insights
- Some discussion and critical analysis
- Conclusions and further work
- Rreferences

## Checklist for analysis

#### screening and cleaning data
- Missing values
- Trim extra words (e.g new)

#### Exploratory Analysis
- Assumptions about the data such as distribution
- Mean
- Median
- Mode
- Standard deviation

#### Main analysis
- Most viewed articles
- Most cited articles 
- which topics publish the most
- greatest impact factor
- most popular fields
- sentiemnt analysis
    - eg article titles/abstracts/conclusions
- What are the most common topics?
- compare article similarity with cosine similarity 



## Data Aquisition

#### All Journals
- titles
- total articles
- total citations
- total article views
- impact factor
- cite score